In [1]:
import os
import cv2
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import numpy as np
import pickle
import matplotlib.pyplot as plt

from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
from torchsummary import summary

# Pollen-Detection-Network

### Network

In [2]:
class FCNN(nn.Module):
    """
    This is a class for fully convolutional neural networks.
    
    It is a subclass of the Module class from torch.nn.
    See the torch.nn documentation for more information.
    """
    
    def __init__(self):
        """
        The constructor for FCNN class. The internal states of the network are initialized. 
        """
        
        super(FCNN, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=30, out_channels=30, kernel_size=5, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1)
        self.conv2_drop = nn.Dropout2d()
        
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=60, kernel_size=3, stride=2)
        self.conv3_drop = nn.Dropout2d()
        
        self.conv4 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=3, stride=1)
        
        self.conv6 = nn.Conv2d(in_channels=120, out_channels=1, kernel_size=1, stride=1)
        
        
    def forward(self, data):
        """
        Defines the computation performed at every call.
        
        Parameters:
            data (torch.Tensor): The input that is evaluated by the network. 
                The network expects the input to be of 4 dimensions.
            
        Returns:
            x (torch.Tensor): The output of the network after evaluating it on the given input.
        """
        
        x = F.relu(self.conv0(data))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = F.relu(self.conv3_drop(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.sigmoid(self.conv6(x))
        return x

### Validation

In [3]:
def validate(network, testloader):
    """
    Calculates the mean validation loss, the F1-Score and the accuracy of the given network on the testloader data.
    
    Parameters:
        network (nn.Module): The network to validate. 
        testloader (torch.utils.data.DataLoader): Contains the data which is used to validate the network. 
        
    Returns:
        validation_loss (float): The mean loss per image with regards to a fixed criterion.
        F1_Score (float): The harmonic mean of the precision and the recall of the given network
            on the given validation data.
        correct / total (float): The percentage of correctly classified samples of the 
            total number of validation samples.
        
    """
    
    total = 0
    correct = 0
    true_positive = 0
    false_negative = 0
    false_positive = 0
    true_negative = 0

    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    with torch.no_grad():
        validation_loss = 0
        for i, (inputs, labels) in enumerate(testloader):
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            loss = criterion(outputs,labels)
            validation_loss += loss.item()
            predicted = (outputs >= threshold) # Predicted is a tensor of booleans 
            total += labels.size(0)
            predicted = predicted.view(predicted.size(0)) 
            labels = labels == 1
            correct += (predicted == labels).sum().item()
            true_positive += (predicted & labels).sum().item()
            false_negative += (np.logical_not(np.array(torch.Tensor.cpu(predicted))) & np.array(torch.Tensor.cpu(labels))).sum()
            
        validation_loss = validation_loss / total 
        true_negative = correct - true_positive
        false_positive = total - correct - false_negative
        
        try: 
            recall = true_positive / (true_positive + false_negative)
        except ZeroDivisionError:
            recall = 0
        try:
            precision = true_positive / (true_positive + false_positive)
        except ZeroDivisionError:
            precision = 0
        try:
            F1_score = 2 * (precision * recall) / (precision + recall)
        except ZeroDivisionError:
            F1_score = 0
            
    
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
    print('F1 Score of the network on the test images: %f' % F1_score)
    return validation_loss, F1_score, (correct / total)

### Training

In [4]:
def train(network, trainloader, ep, criterion, optimizer, print_interval):
    """
    Train a neural network.
    
    
    Parameters:
        network (nn.Module): The network which should be trained.
        trainloader (torch.utils.data.DataLoader): Contains the data to be used to train the network.
        ep (int): The number of epochs the network will be trained for.
        criterion (): The loss function used for training (should be the same for validation).
        optimizer (): The specification how the weights are optimized.
        print_interval (int): The number of batches after which the function prints the loss for one batch.
    
    Returns:
        train_losses ([float]): The mean of the loss for one train sample after every epoch. 
        validation_losses ([float]): The mean of the loss for one validation sample after every epoch.
        F1 ([float]): The F1-Score of the validation data after every epoch.
        accuracy ([float]): The accuracy of the network on the validation data after every epoch.
    """
    
    losses = [] # Mean of loss/image in every epoch of training 
    validation_losses = []
    F1 = []
    accuracy = []
    print('Performance of the untrained network:')
    validate(network, validloader)
    
    for epoch in range(ep):

        loss_stats = 0.0
        total = 0
        
        running_loss = 0.0
    
        for i, (inputs, lables) in enumerate(trainloader):
            
            inputs, lables = inputs.to(device), lables.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            #print(lables.shape)
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            #print(outputs.shape)
            loss = criterion(outputs, lables)
            loss.backward() #propagate the error back through the network
            optimizer.step() #adjust the weights of the network depending on the propagated error
    
            #that's it.
            total += lables.shape[0]
            
            #Some statistics:
            running_loss += loss.item()
            
            loss_stats += loss.item()
            
            if i % print_interval == print_interval - 1:    # print every x mini-batches (loss for one batch)
                print('[Epoch: %d, Batch: %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_interval))
                running_loss = 0.0
        
        losses.append(loss_stats / total) 
        #Test the network using the validation set after every epoch of training.
        val_loss_curr, F1_curr, accuracy_curr = validate(network, validloader)
        validation_losses.append(val_loss_curr)
        F1.append(F1_curr)
        accuracy.append(accuracy_curr)
        
    print('Finished Training')
    return losses, validation_losses, F1, accuracy

### Loading Data

In [5]:
def load_pollen_data(dir):
    np_pollen_data = []     # single date will be [img,label]
    for folder in next(os.walk(dir))[1]:
        if folder == 'p':
            label = 1
            skip = True
        else:
            label = 0
            skip = False
        parent_path = os.path.join(dir, folder)
        for i, file in enumerate(os.listdir(parent_path)):
            if skip:
                if i % 9 != 1: 
                    continue
            if '.png' in file:
                try:
                    path = os.path.join(parent_path, file)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    np_pollen_data.append([np.array(img), label])
                except Exception as e:
                    print(folder, file, str(e))

    data = np.random.shuffle(np_pollen_data)
    print("Data created.")
    return np_pollen_data

### Create Train and Testsets

In [6]:
def create_test_val_sets(data, ratio=0.8):
    """
    Splits the data into a training and a validation data set.
    
    Parameters:
        data ([[obj, obj]]): A list of lists, which contain a sample and the correponding label.
        ratio (float): The ratio between the size of the validation and the training data set.
        
    Returns:
        train_x ([obj]): The samples of the training data set.
        train_y ([obj]): The corresponding labels of the training data set.
        valid_x ([obj]): The samples of the validation data set.
        valid_y ([obj]): The corresponding labels of the validation data set.
    
    """
    images = []
    labels = []

    for i in range(len(data)):
        images.append(data[i][0])
        labels.append(data[i][1])
    
    train_x = images[:int(len(images)*ratio)]
    train_y = labels[:int(len(images)*ratio)]
    valid_x = images[int(len(images)*ratio):]
    valid_y = labels[int(len(images)*ratio):]
    
    return train_x, train_y, valid_x, valid_y

# Train Network

#### Load Data

In [7]:
# Load Data
cur_dir = os.getcwd()
pollen_path = cur_dir + '/Datasets/PollenData64/'
pollendata = load_pollen_data(pollen_path)


Data created.


In [8]:
# Split into sets
train_x, train_y, valid_x, valid_y = create_test_val_sets(pollendata, 0.8)

pollendata = []

# Our neural network expects a dataloader
trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(train_x),torch.Tensor(train_y)),batch_size=32)
validloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(valid_x),torch.Tensor(valid_y)),batch_size=100)

# Save dataloader for later use
#torch.save(trainloader, 'trainloader.pth')
#torch.save(validloader, 'validloader.pth')

#### Initialise the network

In [9]:
# Setup wether gpu or cpu should be used for computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a new instance of the FCNN class. This will be our network.      
pollen_network = FCNN().to(device)

# Choose optimizer, criterion, number of epochs and threshold for the network to accept or dismiss a computated input
criterion = nn.MSELoss()
optimizer = optim.SGD(pollen_network.parameters(), lr=0.001, momentum=0.9)
ep = 10
threshold = 0.8



/usr/local/lib/python3.7/site-packages/torch/cuda/__init__.py:118: UserWarning: 
    Found GPU0 GeForce GT 650M which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


#### Now we can train our network with given data

In [10]:
train_losses, val_losses, F1_scores, accuracies = train(pollen_network, trainloader, ep, criterion, optimizer, 32)

Performance of the untrained network:
Accuracy of the network on the test images: 82 %
F1 Score of the network on the test images: 0.006543
[Epoch: 1, Batch:    32] loss: 0.130
[Epoch: 1, Batch:    64] loss: 0.093
[Epoch: 1, Batch:    96] loss: 0.094
[Epoch: 1, Batch:   128] loss: 0.109
[Epoch: 1, Batch:   160] loss: 0.093
[Epoch: 1, Batch:   192] loss: 0.088
[Epoch: 1, Batch:   224] loss: 0.093
[Epoch: 1, Batch:   256] loss: 0.094
[Epoch: 1, Batch:   288] loss: 0.089
[Epoch: 1, Batch:   320] loss: 0.101
[Epoch: 1, Batch:   352] loss: 0.077
[Epoch: 1, Batch:   384] loss: 0.102
[Epoch: 1, Batch:   416] loss: 0.093
[Epoch: 1, Batch:   448] loss: 0.093
[Epoch: 1, Batch:   480] loss: 0.111
[Epoch: 1, Batch:   512] loss: 0.104
[Epoch: 1, Batch:   544] loss: 0.102
[Epoch: 1, Batch:   576] loss: 0.087
[Epoch: 1, Batch:   608] loss: 0.113
[Epoch: 1, Batch:   640] loss: 0.110
[Epoch: 1, Batch:   672] loss: 0.084
[Epoch: 1, Batch:   704] loss: 0.094
[Epoch: 1, Batch:   736] loss: 0.119
[Epoch: 1

[Epoch: 2, Batch:   864] loss: 0.028
[Epoch: 2, Batch:   896] loss: 0.032
[Epoch: 2, Batch:   928] loss: 0.024
[Epoch: 2, Batch:   960] loss: 0.021
[Epoch: 2, Batch:   992] loss: 0.029
[Epoch: 2, Batch:  1024] loss: 0.029
[Epoch: 2, Batch:  1056] loss: 0.026
[Epoch: 2, Batch:  1088] loss: 0.022
[Epoch: 2, Batch:  1120] loss: 0.022
[Epoch: 2, Batch:  1152] loss: 0.027
[Epoch: 2, Batch:  1184] loss: 0.019
[Epoch: 2, Batch:  1216] loss: 0.025
[Epoch: 2, Batch:  1248] loss: 0.019
[Epoch: 2, Batch:  1280] loss: 0.036
[Epoch: 2, Batch:  1312] loss: 0.023
[Epoch: 2, Batch:  1344] loss: 0.029
[Epoch: 2, Batch:  1376] loss: 0.021
[Epoch: 2, Batch:  1408] loss: 0.022
[Epoch: 2, Batch:  1440] loss: 0.021
[Epoch: 2, Batch:  1472] loss: 0.026
[Epoch: 2, Batch:  1504] loss: 0.024
[Epoch: 2, Batch:  1536] loss: 0.031
[Epoch: 2, Batch:  1568] loss: 0.033
[Epoch: 2, Batch:  1600] loss: 0.022
[Epoch: 2, Batch:  1632] loss: 0.026
[Epoch: 2, Batch:  1664] loss: 0.030
[Epoch: 2, Batch:  1696] loss: 0.027
[

[Epoch: 3, Batch:  1824] loss: 0.027
[Epoch: 3, Batch:  1856] loss: 0.027
[Epoch: 3, Batch:  1888] loss: 0.017
[Epoch: 3, Batch:  1920] loss: 0.029
[Epoch: 3, Batch:  1952] loss: 0.025
[Epoch: 3, Batch:  1984] loss: 0.022
[Epoch: 3, Batch:  2016] loss: 0.026
[Epoch: 3, Batch:  2048] loss: 0.020
[Epoch: 3, Batch:  2080] loss: 0.021
[Epoch: 3, Batch:  2112] loss: 0.024
[Epoch: 3, Batch:  2144] loss: 0.021
[Epoch: 3, Batch:  2176] loss: 0.024
[Epoch: 3, Batch:  2208] loss: 0.017
[Epoch: 3, Batch:  2240] loss: 0.021
[Epoch: 3, Batch:  2272] loss: 0.020
[Epoch: 3, Batch:  2304] loss: 0.019
[Epoch: 3, Batch:  2336] loss: 0.017
[Epoch: 3, Batch:  2368] loss: 0.027
[Epoch: 3, Batch:  2400] loss: 0.022
[Epoch: 3, Batch:  2432] loss: 0.022
[Epoch: 3, Batch:  2464] loss: 0.020
[Epoch: 3, Batch:  2496] loss: 0.020
[Epoch: 3, Batch:  2528] loss: 0.022
[Epoch: 3, Batch:  2560] loss: 0.023
[Epoch: 3, Batch:  2592] loss: 0.020
[Epoch: 3, Batch:  2624] loss: 0.024
[Epoch: 3, Batch:  2656] loss: 0.013
[

[Epoch: 4, Batch:  2784] loss: 0.015
[Epoch: 4, Batch:  2816] loss: 0.019
[Epoch: 4, Batch:  2848] loss: 0.025
[Epoch: 4, Batch:  2880] loss: 0.019
[Epoch: 4, Batch:  2912] loss: 0.025
[Epoch: 4, Batch:  2944] loss: 0.023
[Epoch: 4, Batch:  2976] loss: 0.017
[Epoch: 4, Batch:  3008] loss: 0.024
[Epoch: 4, Batch:  3040] loss: 0.015
[Epoch: 4, Batch:  3072] loss: 0.014
[Epoch: 4, Batch:  3104] loss: 0.025
[Epoch: 4, Batch:  3136] loss: 0.022
[Epoch: 4, Batch:  3168] loss: 0.027
[Epoch: 4, Batch:  3200] loss: 0.020
[Epoch: 4, Batch:  3232] loss: 0.019
[Epoch: 4, Batch:  3264] loss: 0.021
[Epoch: 4, Batch:  3296] loss: 0.017
[Epoch: 4, Batch:  3328] loss: 0.022
[Epoch: 4, Batch:  3360] loss: 0.015
[Epoch: 4, Batch:  3392] loss: 0.013
[Epoch: 4, Batch:  3424] loss: 0.022
[Epoch: 4, Batch:  3456] loss: 0.011
[Epoch: 4, Batch:  3488] loss: 0.028
[Epoch: 4, Batch:  3520] loss: 0.019
[Epoch: 4, Batch:  3552] loss: 0.021
[Epoch: 4, Batch:  3584] loss: 0.028
[Epoch: 4, Batch:  3616] loss: 0.018
[

[Epoch: 5, Batch:  3744] loss: 0.025
[Epoch: 5, Batch:  3776] loss: 0.024
[Epoch: 5, Batch:  3808] loss: 0.017
[Epoch: 5, Batch:  3840] loss: 0.018
[Epoch: 5, Batch:  3872] loss: 0.016
[Epoch: 5, Batch:  3904] loss: 0.021
[Epoch: 5, Batch:  3936] loss: 0.025
[Epoch: 5, Batch:  3968] loss: 0.019
[Epoch: 5, Batch:  4000] loss: 0.019
[Epoch: 5, Batch:  4032] loss: 0.021
[Epoch: 5, Batch:  4064] loss: 0.023
[Epoch: 5, Batch:  4096] loss: 0.016
[Epoch: 5, Batch:  4128] loss: 0.019
[Epoch: 5, Batch:  4160] loss: 0.015
[Epoch: 5, Batch:  4192] loss: 0.015
[Epoch: 5, Batch:  4224] loss: 0.021
[Epoch: 5, Batch:  4256] loss: 0.016
[Epoch: 5, Batch:  4288] loss: 0.022
[Epoch: 5, Batch:  4320] loss: 0.020
[Epoch: 5, Batch:  4352] loss: 0.019
[Epoch: 5, Batch:  4384] loss: 0.020
[Epoch: 5, Batch:  4416] loss: 0.017
[Epoch: 5, Batch:  4448] loss: 0.015
[Epoch: 5, Batch:  4480] loss: 0.018
[Epoch: 5, Batch:  4512] loss: 0.020
[Epoch: 5, Batch:  4544] loss: 0.017
[Epoch: 5, Batch:  4576] loss: 0.019
[

[Epoch: 6, Batch:  4704] loss: 0.016
[Epoch: 6, Batch:  4736] loss: 0.018
[Epoch: 6, Batch:  4768] loss: 0.020
[Epoch: 6, Batch:  4800] loss: 0.020
[Epoch: 6, Batch:  4832] loss: 0.017
[Epoch: 6, Batch:  4864] loss: 0.018
[Epoch: 6, Batch:  4896] loss: 0.020
[Epoch: 6, Batch:  4928] loss: 0.019
[Epoch: 6, Batch:  4960] loss: 0.012
[Epoch: 6, Batch:  4992] loss: 0.018
[Epoch: 6, Batch:  5024] loss: 0.024
[Epoch: 6, Batch:  5056] loss: 0.015
[Epoch: 6, Batch:  5088] loss: 0.013
[Epoch: 6, Batch:  5120] loss: 0.018
[Epoch: 6, Batch:  5152] loss: 0.010
[Epoch: 6, Batch:  5184] loss: 0.021
[Epoch: 6, Batch:  5216] loss: 0.019
[Epoch: 6, Batch:  5248] loss: 0.020
[Epoch: 6, Batch:  5280] loss: 0.020
[Epoch: 6, Batch:  5312] loss: 0.021
[Epoch: 6, Batch:  5344] loss: 0.017
[Epoch: 6, Batch:  5376] loss: 0.018
[Epoch: 6, Batch:  5408] loss: 0.026
[Epoch: 6, Batch:  5440] loss: 0.017
[Epoch: 6, Batch:  5472] loss: 0.015
[Epoch: 6, Batch:  5504] loss: 0.018
[Epoch: 6, Batch:  5536] loss: 0.019
[

[Epoch: 7, Batch:  5664] loss: 0.017
[Epoch: 7, Batch:  5696] loss: 0.016
[Epoch: 7, Batch:  5728] loss: 0.016
[Epoch: 7, Batch:  5760] loss: 0.018
[Epoch: 7, Batch:  5792] loss: 0.013
[Epoch: 7, Batch:  5824] loss: 0.020
[Epoch: 7, Batch:  5856] loss: 0.021
[Epoch: 7, Batch:  5888] loss: 0.019
[Epoch: 7, Batch:  5920] loss: 0.014
[Epoch: 7, Batch:  5952] loss: 0.020
[Epoch: 7, Batch:  5984] loss: 0.021
[Epoch: 7, Batch:  6016] loss: 0.017
[Epoch: 7, Batch:  6048] loss: 0.017
Accuracy of the network on the test images: 97 %
F1 Score of the network on the test images: 0.854923
[Epoch: 8, Batch:    32] loss: 0.017
[Epoch: 8, Batch:    64] loss: 0.016
[Epoch: 8, Batch:    96] loss: 0.016
[Epoch: 8, Batch:   128] loss: 0.022
[Epoch: 8, Batch:   160] loss: 0.019
[Epoch: 8, Batch:   192] loss: 0.016
[Epoch: 8, Batch:   224] loss: 0.014
[Epoch: 8, Batch:   256] loss: 0.018
[Epoch: 8, Batch:   288] loss: 0.011
[Epoch: 8, Batch:   320] loss: 0.010
[Epoch: 8, Batch:   352] loss: 0.015
[Epoch: 8,

[Epoch: 9, Batch:   480] loss: 0.020
[Epoch: 9, Batch:   512] loss: 0.018
[Epoch: 9, Batch:   544] loss: 0.020
[Epoch: 9, Batch:   576] loss: 0.013
[Epoch: 9, Batch:   608] loss: 0.018
[Epoch: 9, Batch:   640] loss: 0.016
[Epoch: 9, Batch:   672] loss: 0.015
[Epoch: 9, Batch:   704] loss: 0.014
[Epoch: 9, Batch:   736] loss: 0.019
[Epoch: 9, Batch:   768] loss: 0.020
[Epoch: 9, Batch:   800] loss: 0.015
[Epoch: 9, Batch:   832] loss: 0.016
[Epoch: 9, Batch:   864] loss: 0.019
[Epoch: 9, Batch:   896] loss: 0.017
[Epoch: 9, Batch:   928] loss: 0.013
[Epoch: 9, Batch:   960] loss: 0.014
[Epoch: 9, Batch:   992] loss: 0.023
[Epoch: 9, Batch:  1024] loss: 0.016
[Epoch: 9, Batch:  1056] loss: 0.016
[Epoch: 9, Batch:  1088] loss: 0.012
[Epoch: 9, Batch:  1120] loss: 0.011
[Epoch: 9, Batch:  1152] loss: 0.017
[Epoch: 9, Batch:  1184] loss: 0.012
[Epoch: 9, Batch:  1216] loss: 0.015
[Epoch: 9, Batch:  1248] loss: 0.013
[Epoch: 9, Batch:  1280] loss: 0.031
[Epoch: 9, Batch:  1312] loss: 0.019
[

[Epoch: 10, Batch:  1408] loss: 0.015
[Epoch: 10, Batch:  1440] loss: 0.012
[Epoch: 10, Batch:  1472] loss: 0.018
[Epoch: 10, Batch:  1504] loss: 0.013
[Epoch: 10, Batch:  1536] loss: 0.021
[Epoch: 10, Batch:  1568] loss: 0.021
[Epoch: 10, Batch:  1600] loss: 0.015
[Epoch: 10, Batch:  1632] loss: 0.020
[Epoch: 10, Batch:  1664] loss: 0.016
[Epoch: 10, Batch:  1696] loss: 0.018
[Epoch: 10, Batch:  1728] loss: 0.013
[Epoch: 10, Batch:  1760] loss: 0.012
[Epoch: 10, Batch:  1792] loss: 0.021
[Epoch: 10, Batch:  1824] loss: 0.022
[Epoch: 10, Batch:  1856] loss: 0.023
[Epoch: 10, Batch:  1888] loss: 0.011
[Epoch: 10, Batch:  1920] loss: 0.019
[Epoch: 10, Batch:  1952] loss: 0.014
[Epoch: 10, Batch:  1984] loss: 0.012
[Epoch: 10, Batch:  2016] loss: 0.017
[Epoch: 10, Batch:  2048] loss: 0.013
[Epoch: 10, Batch:  2080] loss: 0.017
[Epoch: 10, Batch:  2112] loss: 0.013
[Epoch: 10, Batch:  2144] loss: 0.014
[Epoch: 10, Batch:  2176] loss: 0.018
[Epoch: 10, Batch:  2208] loss: 0.009
[Epoch: 10, 

#### Save trained Network

In [12]:
# save the networks parameters to have the possibility to reload them later:
cur_dir = os.getcwd()
save_path = cur_dir + '/pollennet.pt'
torch.save(pollen_network.state_dict(), save_path)

# save the corresponding losses, F1-Scores and Accuracies using pickle:

with open('train_losses_gpu.obj', 'wb') as train_losses_file:
    pickle.dump(train_losses, train_losses_file)

with open('val_losses_gpu.obj', 'wb') as val_losses_file:
    pickle.dump(val_losses, val_losses_file)
    
with open('F1_gpu.obj', 'wb') as F1_file:
    pickle.dump(F1_scores, F1_file)
    
with open('accuracies_gpu.obj', 'wb') as accuracies_file:
    pickle.dump(accuracies, accuracies_file)

In [ ]:
train_losses_2, val_losses_2, F1_scores_2, accuracies_2 = train(pollen_network, trainloader, ep, criterion, optimizer, 32)

Performance of the untrained network:
Accuracy of the network on the test images: 97 %
F1 Score of the network on the test images: 0.879973
[Epoch: 1, Batch:    32] loss: 0.018
[Epoch: 1, Batch:    64] loss: 0.017
[Epoch: 1, Batch:    96] loss: 0.013
[Epoch: 1, Batch:   128] loss: 0.017
[Epoch: 1, Batch:   160] loss: 0.017
[Epoch: 1, Batch:   192] loss: 0.016
[Epoch: 1, Batch:   224] loss: 0.011
[Epoch: 1, Batch:   256] loss: 0.015
[Epoch: 1, Batch:   288] loss: 0.011
[Epoch: 1, Batch:   320] loss: 0.011
[Epoch: 1, Batch:   352] loss: 0.013
[Epoch: 1, Batch:   384] loss: 0.018
[Epoch: 1, Batch:   416] loss: 0.019
[Epoch: 1, Batch:   448] loss: 0.017
[Epoch: 1, Batch:   480] loss: 0.019
[Epoch: 1, Batch:   512] loss: 0.014
[Epoch: 1, Batch:   544] loss: 0.017
[Epoch: 1, Batch:   576] loss: 0.015
[Epoch: 1, Batch:   608] loss: 0.016
[Epoch: 1, Batch:   640] loss: 0.020
[Epoch: 1, Batch:   672] loss: 0.012
[Epoch: 1, Batch:   704] loss: 0.014
[Epoch: 1, Batch:   736] loss: 0.020
[Epoch: 1

[Epoch: 2, Batch:   864] loss: 0.018
[Epoch: 2, Batch:   896] loss: 0.019
[Epoch: 2, Batch:   928] loss: 0.012
[Epoch: 2, Batch:   960] loss: 0.012
[Epoch: 2, Batch:   992] loss: 0.019
[Epoch: 2, Batch:  1024] loss: 0.015
[Epoch: 2, Batch:  1056] loss: 0.015
[Epoch: 2, Batch:  1088] loss: 0.013
[Epoch: 2, Batch:  1120] loss: 0.011
[Epoch: 2, Batch:  1152] loss: 0.015
[Epoch: 2, Batch:  1184] loss: 0.011
[Epoch: 2, Batch:  1216] loss: 0.013
[Epoch: 2, Batch:  1248] loss: 0.012
[Epoch: 2, Batch:  1280] loss: 0.025
[Epoch: 2, Batch:  1312] loss: 0.015
[Epoch: 2, Batch:  1344] loss: 0.015
[Epoch: 2, Batch:  1376] loss: 0.012
[Epoch: 2, Batch:  1408] loss: 0.013
[Epoch: 2, Batch:  1440] loss: 0.011
[Epoch: 2, Batch:  1472] loss: 0.020
[Epoch: 2, Batch:  1504] loss: 0.010
[Epoch: 2, Batch:  1536] loss: 0.018
[Epoch: 2, Batch:  1568] loss: 0.016
